### Procesamiento dataset de erupciones volcanicas

En primer lugar, importamos las librerias necesarias

In [29]:
import csv
import os.path
import os
import requests
import traductor

Cabe destacar que se ha importado la libreria *traductor* para poder realizar la traduccion de palabras del ingles al espanol. La misma se puede consultar en el siguiente [link](https://github.com/LuisAlejandroSalcedo/traductor-de-palabras/blob/master/traductor.py).

A continuacion, abrimos el archivo y guardamos los datos del mismo. en *header* se almacenaran los titulos de las columnas, y en *erupciones* los datos de cada una 

In [30]:
path_files = "Dataset"
archivo_erupciones_volcanicas = "significant-volcanic-eruption-database.csv"
path_arch = os.path.join(os.getcwd(), path_files)
try:
    with open(os.path.join(path_arch, archivo_erupciones_volcanicas)) as erupciones_volcanicas:
        data_erupciones = csv.reader(erupciones_volcanicas, delimiter=';')
        header , erupciones = next(data_erupciones), list(data_erupciones)
except FileNotFoundError as no_hay_archivo:
    print(f'El archivo a procesar no se encuentra en la ruta especificada, error: {no_hay_archivo}')        

Se definen las funciones que se utilizara para el procesado de los datos del archivo de acuerdo a lo solicitado

In [31]:
def generar_traducciones(data_arch):
    """
        Funcion que encuentra todos los diferentes tipos de volcanes y las traduce del ingles al español
        Devuelve las traducciones en un diccionario, donde cada clave es la palabra en ingles y los valores son sus traducciones
    """
    lista_tipos = map(lambda fila: fila[9], data_arch)
    tipos_volcanes = list(set(lista_tipos))
    dicci_traducciones = {tipo:traductor.Traduccion("en","es",tipo) for tipo in tipos_volcanes}
    return dicci_traducciones

traducciones = generar_traducciones(erupciones)

header.append("Name and country")

def combinar_nombre_volcan_y_pais(una_linea):
    """
        Funcion que recibe una linea del archivo y concatena los strings al formato nombre_volcan(pais_al_que_pertenece)
    """
    return una_linea[5] + "(" + una_linea[7] + ")"

def comprobar_flag(una_linea, indice, cadena_a_comprobar):
    """
        Funcion que convierte a valores booleanos los datos contenidos en las columnas flag earthquake y flag tsunami.
    """
    if una_linea[indice] == cadena_a_comprobar:
        una_linea[indice] = True
    else:
        una_linea[indice] = False    


Procesamos cada linea del archivo realizando lo solicitado, invocando a cada funcion correspondiente

In [32]:
for linea in erupciones:
    linea[9] = traducciones[linea[9]]
    linea.append(combinar_nombre_volcan_y_pais(linea))
    comprobar_flag(linea, 3, "Tsunami")
    comprobar_flag(linea, 4, "Earthquake")


Una vez procesada cada linea del archivo, se procede a guardar los datos en un archivo csv, quedandonos solamente con las columnas solicitadas.

In [33]:
nuevo_csv = "erupciones_volcanicas_data.csv"
with open(os.path.join(path_arch, nuevo_csv), 'w', encoding = "utf-8") as salida:
    writer = csv.writer(salida)
    writer.writerow(["Year",  "Volcanic Explosivity Index", "Volcano Type", "Flag Tsunami", "Flag Earthquake", "Name and country"])
    for linea in erupciones:
        writer.writerow([linea[header.index("Year")], linea[header.index("Volcanic Explosivity Index")], linea[header.index("Volcano Type")], linea[header.index("Flag Tsunami")], linea[header.index("Flag Earthquake")], linea[header.index("Name and country")]])